In [1]:
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import csv

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
name_dataframe = pd.read_csv('../data/features.txt', delimiter = '\n', header = None)
names = name_dataframe.values.tolist()
names = [k for row in names for k in row] #List of column names

data = pd.read_csv('../data/X_train.txt', delim_whitespace = True, header = None) #Read in train dataframe
data.columns = names #Setting column names

X_train = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z'] #Selecting only acceleration columns

y_train_activity = pd.read_csv('../data/y_train.txt', header = None)
y_train_activity.columns = ['Activity']

y_train_subject = pd.read_csv('../data/subject_train.txt', header = None)
y_train_subject.columns = ['Subject']

GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
GAN_data.head()

,1 tBodyAcc-mean()-X,2 tBodyAcc-mean()-Y,3 tBodyAcc-mean()-Z,4 tBodyAcc-std()-X,5 tBodyAcc-std()-Y,6 tBodyAcc-std()-Z,7 tBodyAcc-mad()-X,8 tBodyAcc-mad()-Y,9 tBodyAcc-mad()-Z,10 tBodyAcc-max()-X,...,"33 tBodyAcc-arCoeff()-Y,4","34 tBodyAcc-arCoeff()-Z,1","35 tBodyAcc-arCoeff()-Z,2","36 tBodyAcc-arCoeff()-Z,3","37 tBodyAcc-arCoeff()-Z,4","38 tBodyAcc-correlation()-X,Y","39 tBodyAcc-correlation()-X,Z","40 tBodyAcc-correlation()-Y,Z",Activity,Subject
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.095246,0.278851,-0.465085,0.491936,-0.190884,0.376314,0.435129,0.660790,5,1
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.281211,0.085988,-0.022153,-0.016657,-0.220643,-0.013429,-0.072692,0.579382,5,1
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.332564,0.239281,-0.136204,0.173863,-0.299493,-0.124698,-0.181105,0.608900,5,1
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.170813,0.294938,-0.306081,0.482148,-0.470129,-0.305693,-0.362654,0.507459,5,1
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.315375,0.439744,-0.269069,0.179414,-0.088952,-0.155804,-0.189763,0.599213,5,1


In [4]:
def get_train_test_split(data, current_user_tested):
    """
    data: DataFrame
    current_user_tested: int
    
    Returns: numpy arrays of X_train, y_train, X_test, y_test
    """
    data = data.copy(deep = True)
    
    #Getting only acceleration columns
    X_train = data[data['Subject'] == current_user_tested].loc[:, "1 tBodyAcc-mean()-X": "40 tBodyAcc-correlation()-Y,Z"].values
    y_train = data[data['Subject'] == current_user_tested].loc[:, "Activity"].values
    
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.15, shuffle = True)
    
    #Zero indexing all activity labels since they start at 1
    y_train -= 1 
    y_test -=1

    return X_train, y_train, X_test, y_test

In [5]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = 40):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 6)
        )
    def forward(self, x):
        return self.network(x)

In [6]:
def initialize_params(X_train, y_train, X_test, y_test):
    lr = 0.001
    batch_size = 100
    
    model = Classifier().to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = lr)

    train_features = torch.tensor(X_train).to(device)
    train_labels = torch.tensor(y_train).to(device)
    test_features = torch.tensor(X_test).to(device)
    test_labels = torch.tensor(y_test).to(device)
    
    train_data = torch.utils.data.TensorDataset(train_features, train_labels)
    test_data = torch.utils.data.TensorDataset(test_features, test_labels)

    train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)
    
    return model, train_loader, test_loader, optimizer, criterion

def training_loop(model, train_loader, test_loader, optimizer, criterion, sub_num, n_epochs = 1000):
    for epoch in range(n_epochs):
        total_loss = 0
        for batch in train_loader:
            features, labels = batch

            optimizer.zero_grad()
            preds = model(features.float())

            loss = criterion(preds, labels) 
            loss.backward()

            optimizer.step()
            total_loss += loss.item()

        print(f'Subject {sub_num}, Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')
    
    return model, test_loader

In [7]:
def evaluation(model, test_loader):
    softmax = nn.Softmax(dim = 1)
    for batch in test_loader: #Runs once since the batch is the entire testing data
        features, labels = batch
        _, preds = torch.max(softmax(model(features.float())), dim = 1) #Getting the model's predictions
        report = metrics.classification_report(labels.cpu(), preds.cpu(), digits = 3, output_dict = True)
        f1_score = pd.DataFrame(report).transpose().loc['weighted avg', :]['f1-score']
    return f1_score

In [8]:
subject_numbers = list(GAN_data['Subject'].unique()) #list of all unique subject numbers
n_subjects = len(subject_numbers) #Number of unique subjects
n_iters = 100

In [9]:
all_scores = np.zeros((n_subjects, n_iters + 1))

for k in range(n_subjects):
    #Get train/test data of the current user
    X_train, y_train, X_test, y_test = get_train_test_split(GAN_data, subject_numbers[k])
    #Get untrained model, data loaders of current user
    model, train_loader, test_loader, optimizer, criterion = initialize_params(X_train, y_train, X_test, y_test)
    #Get trained model on the current user
    model, test_loader = training_loop(model, train_loader, test_loader, optimizer, criterion, k)
    for j in range(n_iters):
        #Evaluate model on current user
        f1_score = evaluation(model, test_loader)
        all_scores[k, j] = f1_score

Subject 0, Epoch 1, Loss: 5.456624269485474, Final Batch Loss: 1.819804072380066
Subject 0, Epoch 2, Loss: 5.4452574253082275, Final Batch Loss: 1.828585147857666
Subject 0, Epoch 3, Loss: 5.432973623275757, Final Batch Loss: 1.819122314453125
Subject 0, Epoch 4, Loss: 5.412111163139343, Final Batch Loss: 1.8036606311798096
Subject 0, Epoch 5, Loss: 5.405317544937134, Final Batch Loss: 1.8095568418502808
Subject 0, Epoch 6, Loss: 5.400514006614685, Final Batch Loss: 1.8025137186050415
Subject 0, Epoch 7, Loss: 5.380220293998718, Final Batch Loss: 1.762546181678772
Subject 0, Epoch 8, Loss: 5.363085031509399, Final Batch Loss: 1.7890852689743042
Subject 0, Epoch 9, Loss: 5.33820903301239, Final Batch Loss: 1.7749247550964355
Subject 0, Epoch 10, Loss: 5.320154070854187, Final Batch Loss: 1.755284070968628
Subject 0, Epoch 11, Loss: 5.289071321487427, Final Batch Loss: 1.7397757768630981
Subject 0, Epoch 12, Loss: 5.2653738260269165, Final Batch Loss: 1.745194435119629
Subject 0, Epoch 1

Subject 0, Epoch 114, Loss: 1.914885401725769, Final Batch Loss: 0.6398683786392212
Subject 0, Epoch 115, Loss: 1.793815553188324, Final Batch Loss: 0.543848991394043
Subject 0, Epoch 116, Loss: 1.8201097249984741, Final Batch Loss: 0.6440517902374268
Subject 0, Epoch 117, Loss: 1.7998219728469849, Final Batch Loss: 0.6323148012161255
Subject 0, Epoch 118, Loss: 1.8430272340774536, Final Batch Loss: 0.6041670441627502
Subject 0, Epoch 119, Loss: 1.7915604710578918, Final Batch Loss: 0.6110338568687439
Subject 0, Epoch 120, Loss: 1.736860990524292, Final Batch Loss: 0.6020115613937378
Subject 0, Epoch 121, Loss: 1.637149155139923, Final Batch Loss: 0.5861057043075562
Subject 0, Epoch 122, Loss: 1.7026094794273376, Final Batch Loss: 0.5115756392478943
Subject 0, Epoch 123, Loss: 1.7233060598373413, Final Batch Loss: 0.5487342476844788
Subject 0, Epoch 124, Loss: 1.7409935593605042, Final Batch Loss: 0.5283254384994507
Subject 0, Epoch 125, Loss: 1.6954864859580994, Final Batch Loss: 0.58

Subject 0, Epoch 226, Loss: 1.4219816029071808, Final Batch Loss: 0.42409440875053406
Subject 0, Epoch 227, Loss: 1.3759199976921082, Final Batch Loss: 0.4151027798652649
Subject 0, Epoch 228, Loss: 1.365662306547165, Final Batch Loss: 0.4191722571849823
Subject 0, Epoch 229, Loss: 1.4288081228733063, Final Batch Loss: 0.4228740334510803
Subject 0, Epoch 230, Loss: 1.4014484286308289, Final Batch Loss: 0.4972996115684509
Subject 0, Epoch 231, Loss: 1.3827761709690094, Final Batch Loss: 0.5994147658348083
Subject 0, Epoch 232, Loss: 1.3715060651302338, Final Batch Loss: 0.39278897643089294
Subject 0, Epoch 233, Loss: 1.3475214838981628, Final Batch Loss: 0.3742900490760803
Subject 0, Epoch 234, Loss: 1.3844731450080872, Final Batch Loss: 0.46348294615745544
Subject 0, Epoch 235, Loss: 1.47956183552742, Final Batch Loss: 0.5928741693496704
Subject 0, Epoch 236, Loss: 1.3625699281692505, Final Batch Loss: 0.47842228412628174
Subject 0, Epoch 237, Loss: 1.3900305330753326, Final Batch Loss

Subject 0, Epoch 330, Loss: 0.9304144382476807, Final Batch Loss: 0.3465802073478699
Subject 0, Epoch 331, Loss: 0.9303751885890961, Final Batch Loss: 0.27310094237327576
Subject 0, Epoch 332, Loss: 0.9200699627399445, Final Batch Loss: 0.3206087648868561
Subject 0, Epoch 333, Loss: 0.8874774873256683, Final Batch Loss: 0.2764609754085541
Subject 0, Epoch 334, Loss: 0.8998711407184601, Final Batch Loss: 0.2839709520339966
Subject 0, Epoch 335, Loss: 0.9310825616121292, Final Batch Loss: 0.23601655662059784
Subject 0, Epoch 336, Loss: 0.9483212828636169, Final Batch Loss: 0.3368874490261078
Subject 0, Epoch 337, Loss: 0.9858790040016174, Final Batch Loss: 0.3219295144081116
Subject 0, Epoch 338, Loss: 0.9305860549211502, Final Batch Loss: 0.24732203781604767
Subject 0, Epoch 339, Loss: 0.9378388822078705, Final Batch Loss: 0.353707492351532
Subject 0, Epoch 340, Loss: 0.9401772618293762, Final Batch Loss: 0.3330047130584717
Subject 0, Epoch 341, Loss: 0.9060929417610168, Final Batch Los

Subject 0, Epoch 438, Loss: 0.8226539343595505, Final Batch Loss: 0.24600768089294434
Subject 0, Epoch 439, Loss: 0.7867465317249298, Final Batch Loss: 0.20179292559623718
Subject 0, Epoch 440, Loss: 0.7206668257713318, Final Batch Loss: 0.2306816577911377
Subject 0, Epoch 441, Loss: 0.802410751581192, Final Batch Loss: 0.24672242999076843
Subject 0, Epoch 442, Loss: 0.7260285764932632, Final Batch Loss: 0.23454906046390533
Subject 0, Epoch 443, Loss: 0.7178303003311157, Final Batch Loss: 0.20567665994167328
Subject 0, Epoch 444, Loss: 0.661226436495781, Final Batch Loss: 0.20663510262966156
Subject 0, Epoch 445, Loss: 0.7559047490358353, Final Batch Loss: 0.2727009654045105
Subject 0, Epoch 446, Loss: 0.7511196285486221, Final Batch Loss: 0.20427893102169037
Subject 0, Epoch 447, Loss: 0.7779697924852371, Final Batch Loss: 0.2024991661310196
Subject 0, Epoch 448, Loss: 0.7604808658361435, Final Batch Loss: 0.2773992717266083
Subject 0, Epoch 449, Loss: 0.6733974516391754, Final Batch 

...

Subject 20, Epoch 526, Loss: 1.1049941331148148, Final Batch Loss: 0.3243744671344757
Subject 20, Epoch 527, Loss: 0.9261207282543182, Final Batch Loss: 0.12366281449794769
Subject 20, Epoch 528, Loss: 1.0084002315998077, Final Batch Loss: 0.16942965984344482
Subject 20, Epoch 529, Loss: 1.0386436879634857, Final Batch Loss: 0.19961410760879517
Subject 20, Epoch 530, Loss: 1.0528944283723831, Final Batch Loss: 0.3736205995082855
Subject 20, Epoch 531, Loss: 0.9115379303693771, Final Batch Loss: 0.24900442361831665
Subject 20, Epoch 532, Loss: 0.845007061958313, Final Batch Loss: 0.09042307734489441
Subject 20, Epoch 533, Loss: 0.9326612651348114, Final Batch Loss: 0.21237753331661224
Subject 20, Epoch 534, Loss: 1.0219798684120178, Final Batch Loss: 0.2875082492828369
Subject 20, Epoch 535, Loss: 1.1057410836219788, Final Batch Loss: 0.3856608271598816
Subject 20, Epoch 536, Loss: 0.9311588108539581, Final Batch Loss: 0.1932561844587326
Subject 20, Epoch 537, Loss: 0.931653693318367, F

Subject 20, Epoch 630, Loss: 1.0406809449195862, Final Batch Loss: 0.4526137411594391
Subject 20, Epoch 631, Loss: 0.7918632254004478, Final Batch Loss: 0.20736055076122284
Subject 20, Epoch 632, Loss: 0.6849991083145142, Final Batch Loss: 0.13895359635353088
Subject 20, Epoch 633, Loss: 0.7944235950708389, Final Batch Loss: 0.23483635485172272
Subject 20, Epoch 634, Loss: 0.8590097874403, Final Batch Loss: 0.351603239774704
Subject 20, Epoch 635, Loss: 0.7725168615579605, Final Batch Loss: 0.16772638261318207
Subject 20, Epoch 636, Loss: 0.8498936742544174, Final Batch Loss: 0.25394076108932495
Subject 20, Epoch 637, Loss: 0.6990882456302643, Final Batch Loss: 0.15605343878269196
Subject 20, Epoch 638, Loss: 0.796584889292717, Final Batch Loss: 0.27543434500694275
Subject 20, Epoch 639, Loss: 0.726659283041954, Final Batch Loss: 0.1576818823814392
Subject 20, Epoch 640, Loss: 0.8148255795240402, Final Batch Loss: 0.13004665076732635
Subject 20, Epoch 641, Loss: 0.6384571343660355, Fin

Subject 20, Epoch 737, Loss: 0.6646216586232185, Final Batch Loss: 0.24236875772476196
Subject 20, Epoch 738, Loss: 0.662579633295536, Final Batch Loss: 0.262666255235672
Subject 20, Epoch 739, Loss: 0.6841345727443695, Final Batch Loss: 0.2657565176486969
Subject 20, Epoch 740, Loss: 0.6707337945699692, Final Batch Loss: 0.17888620495796204
Subject 20, Epoch 741, Loss: 0.5898948982357979, Final Batch Loss: 0.04236336797475815
Subject 20, Epoch 742, Loss: 0.6830967739224434, Final Batch Loss: 0.11434803158044815
Subject 20, Epoch 743, Loss: 0.6698699817061424, Final Batch Loss: 0.11090756207704544
Subject 20, Epoch 744, Loss: 0.6300237961113453, Final Batch Loss: 0.057591501623392105
Subject 20, Epoch 745, Loss: 0.6332842409610748, Final Batch Loss: 0.1325320452451706
Subject 20, Epoch 746, Loss: 0.6483563184738159, Final Batch Loss: 0.1184190958738327
Subject 20, Epoch 747, Loss: 0.6147522628307343, Final Batch Loss: 0.14376245439052582
Subject 20, Epoch 748, Loss: 0.6811227053403854,

Subject 20, Epoch 843, Loss: 0.3565152511000633, Final Batch Loss: 0.06646816432476044
Subject 20, Epoch 844, Loss: 0.47312355041503906, Final Batch Loss: 0.04323451966047287
Subject 20, Epoch 845, Loss: 0.49170249700546265, Final Batch Loss: 0.17876030504703522
Subject 20, Epoch 846, Loss: 0.5373286753892899, Final Batch Loss: 0.08994501829147339
Subject 20, Epoch 847, Loss: 0.7506758868694305, Final Batch Loss: 0.21058301627635956
Subject 20, Epoch 848, Loss: 0.556461937725544, Final Batch Loss: 0.12074095755815506
Subject 20, Epoch 849, Loss: 0.3953806720674038, Final Batch Loss: 0.0530773289501667
Subject 20, Epoch 850, Loss: 0.4925357922911644, Final Batch Loss: 0.10142043977975845
Subject 20, Epoch 851, Loss: 0.4680699482560158, Final Batch Loss: 0.14042872190475464
Subject 20, Epoch 852, Loss: 0.7580598145723343, Final Batch Loss: 0.15015798807144165
Subject 20, Epoch 853, Loss: 0.5136668235063553, Final Batch Loss: 0.05874045193195343
Subject 20, Epoch 854, Loss: 0.473506838083

Subject 20, Epoch 940, Loss: 0.5094058178365231, Final Batch Loss: 0.06061476841568947
Subject 20, Epoch 941, Loss: 0.43026573956012726, Final Batch Loss: 0.04943590611219406
Subject 20, Epoch 942, Loss: 0.3632921986281872, Final Batch Loss: 0.04257887229323387
Subject 20, Epoch 943, Loss: 0.3194118104875088, Final Batch Loss: 0.07990914583206177
Subject 20, Epoch 944, Loss: 0.34300709515810013, Final Batch Loss: 0.03640823811292648
Subject 20, Epoch 945, Loss: 0.31671446189284325, Final Batch Loss: 0.023316223174333572
Subject 20, Epoch 946, Loss: 0.5011211708188057, Final Batch Loss: 0.20150279998779297
Subject 20, Epoch 947, Loss: 0.4958079755306244, Final Batch Loss: 0.2258155792951584
Subject 20, Epoch 948, Loss: 0.3934917449951172, Final Batch Loss: 0.11064191162586212
Subject 20, Epoch 949, Loss: 0.32511530444025993, Final Batch Loss: 0.05247778445482254
Subject 20, Epoch 950, Loss: 0.4845297448337078, Final Batch Loss: 0.18361860513687134
Subject 20, Epoch 951, Loss: 0.43861918

In [10]:
for k in range(n_subjects):
    all_scores[k, -1] = subject_numbers[k]
    
all_scores

array([[ 0.84331756,  0.86647924,  0.84280248, ...,  0.8266438 ,
         0.88374882,  1.        ],
       [ 0.94103282,  0.90357936,  0.92115174, ...,  0.92005495,
         0.96120973,  3.        ],
       [ 0.78065481,  0.86352657,  0.80165631, ...,  0.81401602,
         0.83441438,  5.        ],
       ...,
       [ 0.89581819,  0.86434783,  0.82963518, ...,  0.88013136,
         0.83378132, 28.        ],
       [ 0.88596692,  0.88570804,  0.92451923, ...,  0.82289544,
         0.86727624, 29.        ],
       [ 0.89675698,  0.9307266 ,  0.89865739, ...,  0.89657909,
         0.89663572, 30.        ]])

In [16]:
heading = list(np.arange(0, n_iters, 1)) + ['Subject Number']
heading

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 'Subject Number']

In [17]:
with open("../model_outputs/Single User F-1 Scores.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(heading)
    csvwriter.writerows(all_scores)

print("Model results saved to Single User F-1 Scores.csv")

Model results saved to Single User F-1 Scores.csv
